Army Poker Sim
==============

In the army in 2023, we played a game that we (incorrectly) call "_Chinese Poker_".

During those games, several questions arose. This project aims to anser those questions, by simulating larget numbers of random games.

Questions
---------

1. How much does luck play a factor? (vs ideal strategy)
2. Is it better to focus on grouping, or flush/straight?
3. Is a _flush_ unfairly weighted?



In [1]:
# Setup

import pkgutil
from src import headToHead
from src import strategies

# Import strategies for comparison
package = strategies
prefix = package.__name__ + "."

allStrategies = []
for importer, modname, ispkg in pkgutil.iter_modules(package.__path__, prefix):
    allStrategies.append(__import__(modname, fromlist="dummy"))

#allStrategies = [strategies.preferGroupFlushStraight, strategies.poppy, strategies.bobby]

ITERATIONS = 10000

We will put all of our strategies in a _round-robin_ tournament. Each match-up consists of 10000 games.


A win counts as 1 point (regardless of how many columns won).

In [2]:
import itertools
pairings = itertools.combinations(allStrategies, 2)

resultsTable = []

winners = []

def getPrettyName(nm):
    return nm.split(".")[-1]

for pairing in pairings:
    s1 = pairing[0]
    s2 = pairing[1]
    score = headToHead.head2Head(s1, s2, ITERATIONS)
    winner = s2
    loser = s1
    if score > 0:
        winner = s1
        loser = s2

    #print("%s beat %s" % (getPrettyName(winner.__name__), getPrettyName(loser.__name__)))
    winners.append(winner)

    winPercentage = (((ITERATIONS - abs(score)) / 2) + abs(score) ) / ITERATIONS * 100
    resultsTable.append([getPrettyName(winner.__name__), getPrettyName(loser.__name__), abs(winPercentage)])

results = {}
for strat in set(allStrategies):
    results[getPrettyName(strat.__name__)] = winners.count(strat)

Round-Robin Results
-------------------

In [3]:
import pandas as pd
from IPython.display import HTML
df = pd.DataFrame(resultsTable, columns=["Winner", "Loser", "%"]).sort_values('%', ascending=False)
HTML(df.to_html(index=False))

Winner,Loser,%
bobby,chooseLeft,95.560
preferGroupFlushStraight,chooseLeft,95.460
preferGroupsFlush,chooseLeft,95.290
preferGroups,chooseLeft,95.260
poppy,chooseLeft,95.210
preferGroupStraightFlush,chooseLeft,94.930
preferFlushGroups,chooseLeft,92.385
preferStraightFlushGroup,chooseLeft,91.255
preferFlushGroupsNoSwap,chooseLeft,90.040
preferFlushStraightGroup,chooseLeft,89.750


In [4]:
winPercentageTable = {}
for row in resultsTable:
    if row[0] not in winPercentageTable:
        winPercentageTable[row[0]] = 0
    if row[1] not in winPercentageTable:
        winPercentageTable[row[1]] = 0
    
    winPercentageTable[row[0]] += row[2]
    winPercentageTable[row[1]] += 100 - row[2]

orderedWinners = list(set(winners))
orderedWinners.sort(key=lambda strat: winners.count(strat), reverse=True)
winnerStats = [{"strat": getPrettyName(strat.__name__), "count": winners.count(strat)} for strat in orderedWinners]
#winnerStats

for strat in winPercentageTable:
    winPercentageTable[strat] = round(winPercentageTable[strat] / (len(allStrategies)-1), 2)

winPercentageDF = pd.DataFrame([[strat, winPercentageTable[strat]] for strat in winPercentageTable], columns=["Strategy", "%"]).sort_values('%', ascending=False)
HTML(winPercentageDF.to_html(index=False))
    

Strategy,%
bobby,61.11
preferGroupsFlush,59.79
preferGroupFlushStraight,59.54
poppy,58.71
preferGroupStraightFlush,58.12
preferGroups,57.35
preferFlushGroups,51.55
preferFlushGroupsNoSwap,46.26
preferStraightFlushGroup,46.10
preferFlushStraightGroup,45.00


In [5]:
print("The Strategy that won the most games was %s, winning %s%% of its games." % (winPercentageDF.iloc[0][0], winPercentageDF.iloc[0][1]))

print("The strategy that is best against the most opponents is %s, winning against %s out of %s rivals" % (winnerStats[0]["strat"], winnerStats[0]["count"], (len(allStrategies)-1)))

The Strategy that won the most games was bobby, winning 61.11% of its games.
The strategy that is best against the most opponents is bobby, winning against 10 out of 10 rivals
